# Upstream SDK Demo

This notebook demonstrates the comprehensive usage of the Upstream SDK for managing environmental monitoring campaigns, stations, and data publication to CKAN.

## Overview

The Upstream SDK provides a modern, type-safe interface for:
- 🏕️ **Campaign Management**: Creating and managing monitoring campaigns
- 📡 **Station Management**: Setting up monitoring stations with sensors
- 📊 **Data Management**: Uploading sensor data and measurements
- 🌐 **CKAN Integration**: Publishing datasets to CKAN data portals

## Features Demonstrated

- Authentication and client initialization
- Campaign creation and management
- Station setup and configuration
- Data upload with file handling
- CKAN dataset creation and resource management
- Error handling and validation

## Prerequisites

- Valid Upstream account credentials
- Python 3.7+ environment
- Required packages installed (see requirements)

## Installation and Setup

In [61]:
# Install required packages
!pip install -e .
# Import required libraries
import os
import json
import getpass
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, Optional, List

# Import Upstream SDK modules
from upstream.client import UpstreamClient
from upstream.campaigns import CampaignManager
from upstream.stations import StationManager
from upstream.ckan import CKANIntegration
from upstream.exceptions import APIError, ValidationError
from upstream.auth import AuthManager

Obtaining file:///Users/mosorio/repos/tacc/upstream/sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for upstream-sdk (pyproject.toml) ... done
  Created wheel for upstream-sdk: filename=upstream_sdk-1.0.0-0.editable-py3-none-any.whl size=8003 sha256=9a9cdc447bb53712077a593d8bdb927f5dd3ebdd988afc19bc0b0231e85eaa87
  Stored in directory: /private/var/folders/qn/xpsy3ssx5hbbb_ndr2sbt5w80000gn/T/pip-ephem-wheel-cache-sban1wp5/wheels/47/dc/ae/1a3abd774032839edac85dcd8bb9739031dd6ccef29fca9667
Successfully built upstream-sdk
  Attempting uninstall: upstream-sdk
    Found existing installation: upstream-sdk 1.0.0
    Uninstalling upstream-sdk-1.0.0:
      Successfully uninstalled upstream-sdk-1.0.0

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 1. Authentication and Client Setup

First, let's authenticate with the Upstream API and set up our client instances.

In [62]:
# Configuration
BASE_URL = "https://upstream-dso.tacc.utexas.edu/dev"
CKAN_URL = "https://ckan.tacc.utexas.edu"

BASE_URL = 'http://localhost:8000'
CKAN_URL = 'http://ckan.tacc.cloud:5000'

# Get credentials
print("Please enter your Upstream credentials:")
username = input("Username: ")
password = getpass.getpass("Password: ")

# Initialize client
try:
    client = UpstreamClient(
        username=username,
        password=password,
        base_url=BASE_URL,
        ckan_url=CKAN_URL
    )

    # Test authentication
    if client.authenticate():
        print("✅ Authentication successful!")
        print(f"🔗 Connected to: {BASE_URL}")
        print(f"🌐 CKAN URL: {CKAN_URL}")
    else:
        print("❌ Authentication failed!")
        raise Exception("Authentication failed")

except Exception as e:
    print(f"❌ Setup error: {e}")
    raise

Please enter your Upstream credentials:
✅ Authentication successful!
🔗 Connected to: http://localhost:8000
🌐 CKAN URL: http://ckan.tacc.cloud:5000


## 2. Campaign Management

Let's create and manage environmental monitoring campaigns using the CampaignManager.

In [63]:
# Initialize campaign manager
from upstream_api_client.models import CampaignsIn
campaign_manager = CampaignManager(client.auth_manager)

campaing_request : CampaignsIn = CampaignsIn(
    name="Environmental Monitoring Demo 2024",
    description="Demonstration campaign for SDK usage and CKAN integration",
    contact_name="Dr. Jane Smith",
    contact_email="jane.smith@example.edu",
    allocation="TACC",
    start_date=datetime.now(),
    end_date=datetime.now().replace(year=datetime.now().year + 1)
)

# Create a new campaign
print("📊 Creating new campaign...")
try:
    campaign = campaign_manager.create(campaing_request)

    print(f"✅ Campaign created successfully!")
    print(f"   ID: {campaign.id}")
    campaign_id = campaign.id

except ValidationError as e:
    print(f"❌ Validation error: {e}")
except APIError as e:
    print(f"❌ API error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

📊 Creating new campaign...
✅ Campaign created successfully!
   ID: 609


In [64]:
# List existing campaigns
print("📋 Listing existing campaigns...")
try:
    campaigns = campaign_manager.list(limit=10)
    print(f"Found {campaigns.total} campaigns:")
    for camp in campaigns.items[:1]:  # Show first 5
        print(f"  • {camp.id}: {camp.name}")
        print(f"    Description: {camp.description[:100]}...")
        print()

except Exception as e:
    print(f"❌ Error listing campaigns: {e}")

📋 Listing existing campaigns...
Found 2 campaigns:
  • 1: Test Campaign 2024
    Description: A test campaign for development purposes...



In [65]:
# Get campaign details
print(f"📋 Getting campaign details for ID: {campaign_id}")
try:
    campaign_details = campaign_manager.get(str(campaign_id))

    print(f"Campaign Details:")
    print(f"  Name: {campaign_details.name}")
    print(f"  Description: {campaign_details.description}")
    print(f"  Contact: {campaign_details.contact_name} ({campaign_details.contact_email})")
    print(f"  Allocation: {campaign_details.allocation}")
    print(f"  Start Date: {campaign_details.start_date}")
    print(f"  End Date: {campaign_details.end_date}")

except Exception as e:
    print(f"❌ Error getting campaign details: {e}")

📋 Getting campaign details for ID: 609
Campaign Details:
  Name: Environmental Monitoring Demo 2024
  Description: Demonstration campaign for SDK usage and CKAN integration
  Contact: Dr. Jane Smith (jane.smith@example.edu)
  Allocation: TACC
  Start Date: 2025-07-17 09:07:26.136330
  End Date: 2026-07-17 09:07:26.136334


## 3. Station Management

Now let's create monitoring stations within our campaign using the StationManager.

In [66]:
# Initialize station manager
station_manager = StationManager(client.auth_manager)
from upstream_api_client.models import (
    StationCreate,
)
new_station = StationCreate(
    name="Downtown Air Quality Monitor",
    description="Air quality monitoring station in downtown Austin",
    contact_name="Dr. Jane Smith",
    contact_email="jane.smith@example.edu",
    start_date=datetime.now(),
)

# Create a new station
print("📍 Creating new monitoring station...")
try:
    station = station_manager.create(campaign_id=str(campaign_id), station_create=new_station)

    print(f"✅ Station created successfully!")
    print(f"   ID: {station.id}")
    station_id = station.id

except ValidationError as e:
    print(f"❌ Validation error: {e}")
except APIError as e:
    print(f"❌ API error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

📍 Creating new monitoring station...
✅ Station created successfully!
   ID: 479


In [67]:
# List stations in the campaign
print(f"📋 Listing stations in campaign {campaign_id}...")
try:
    stations = station_manager.list(campaign_id=str(campaign_id))

    print(f"Found {stations.total} stations:")
    for station in stations.items:
        print(f"  • {station.id}: {station.name}")
        print(f"    Description: {station.description[:80]}...")
        print()

except Exception as e:
    print(f"❌ Error listing stations: {e}")

📋 Listing stations in campaign 609...
Found 1 stations:
  • 479: Downtown Air Quality Monitor
    Description: Air quality monitoring station in downtown Austin...



## 4. Data Upload

Let's create sample CSV files and upload sensor data using the client.

In [68]:
# Create sample data directory
data_dir = Path("sample_data")
data_dir.mkdir(exist_ok=True)

# Create sample sensors CSV
sensors_csv = data_dir / "sensors.csv"
sensors_data = """alias,variablename,units,postprocess,postprocessscript
temp_01,Air Temperature,°C,false,
humidity_01,Relative Humidity,%,false,
pressure_01,Atmospheric Pressure,hPa,false,
pm25_01,PM2.5 Concentration,μg/m³,true,pm25_calibration
pm10_01,PM10 Concentration,μg/m³,true,pm10_calibration"""

with open(sensors_csv, 'w') as f:
    f.write(sensors_data)

# Create sample measurements CSV
measurements_csv = data_dir / "measurements.csv"
measurements_data = """collectiontime,Lat_deg,Lon_deg,temp_01,humidity_01,pressure_01,pm25_01,pm10_01
2024-01-15T10:00:00,30.2672,-97.7431,22.5,68.2,1013.25,15.2,25.8
2024-01-15T10:05:00,30.2672,-97.7431,22.7,67.8,1013.20,14.8,24.5
2024-01-15T10:10:00,30.2672,-97.7431,22.9,67.5,1013.15,16.1,26.2
2024-01-15T10:15:00,30.2672,-97.7431,23.1,67.2,1013.10,15.5,25.1
2024-01-15T10:20:00,30.2672,-97.7431,23.3,66.9,1013.05,14.9,24.8
2024-01-15T10:25:00,30.2672,-97.7431,23.5,66.5,1013.00,15.7,26.0
2024-01-15T10:30:00,30.2672,-97.7431,23.7,66.2,1012.95,16.2,26.5
2024-01-15T10:35:00,30.2672,-97.7431,23.9,65.9,1012.90,15.3,25.3
2024-01-15T10:40:00,30.2672,-97.7431,24.1,65.6,1012.85,14.6,24.2
2024-01-15T10:45:00,30.2672,-97.7431,24.3,65.3,1012.80,15.8,25.9"""

with open(measurements_csv, 'w') as f:
    f.write(measurements_data)

print(f"📁 Sample data files created:")
print(f"  • Sensors: {sensors_csv} ({sensors_csv.stat().st_size} bytes)")
print(f"  • Measurements: {measurements_csv} ({measurements_csv.stat().st_size} bytes)")

📁 Sample data files created:
  • Sensors: sample_data/sensors.csv (287 bytes)
  • Measurements: sample_data/measurements.csv (728 bytes)


In [69]:
# Upload CSV data
print(f"📤 Uploading sensor data to station {station_id}...")
try:
    upload_result = client.upload_csv_data(
        campaign_id=campaign_id,
        station_id=station_id,
        sensors_file=sensors_csv,
        measurements_file=measurements_csv
    )

    print(f"✅ Data uploaded successfully!")
    print(json.dumps(upload_result['response'], indent=4))
except Exception as e:
    print(f"❌ Upload error: {e}")

📤 Uploading sensor data to station 479...
✅ Data uploaded successfully!
{
    "uploaded_file_sensors stored in memory": true,
    "uploaded_file_measurements stored in memory": true,
    "Total sensors processed": 5,
    "Total measurements added to database": 50,
    "Data Processing time": "0.1 seconds."
}


## 4.5. List Sensors on Station

Let's list all the sensors that were created on our station after the data upload.

In [80]:
# List all sensors on the station
print(f"📡 Listing all sensors on station {station_id}...")
try:
    sensors = client.sensors.list(
        campaign_id=campaign_id,
        station_id=station_id
    )

    print(f"Found {len(sensors.items)} sensors:")
    for sensor in sensors.items:
        print(f"  • {sensor.alias} ({sensor.variablename})")
        print(f"    Units: {sensor.units}")
        print(f"    Post-process: {sensor.postprocess}")
        if sensor.postprocessscript:
            print(f"    Post-process script: {sensor.postprocessscript}")
        print()

except Exception as e:
    print(f"❌ Error listing sensors: {e}")

📡 Listing all sensors on station 479...
Found 5 sensors:
  • temp_01 (Air Temperature)
    Units: °C
    Post-process: False

  • humidity_01 (Relative Humidity)
    Units: %
    Post-process: False

  • pressure_01 (Atmospheric Pressure)
    Units: hPa
    Post-process: False

  • pm25_01 (PM2.5 Concentration)
    Units: μg/m³
    Post-process: True
    Post-process script: pm25_calibration

  • pm10_01 (PM10 Concentration)
    Units: μg/m³
    Post-process: True
    Post-process script: pm10_calibration



## 5. CKAN Integration

Now let's demonstrate the CKAN integration by publishing our campaign data to a CKAN portal.

In [71]:
# Initialize CKAN integration
print("🌐 Initializing CKAN integration...")
try:
    # Configure CKAN with API key (if available)
    ckan_config = {
        'api_key': os.getenv('CKAN_API_KEY'),  # Set this environment variable
        'timeout': 60,
        'default_organization': 'upstream-environmental-data'
    }

    ckan = CKANIntegration(ckan_url=CKAN_URL, config=ckan_config)

    print(f"✅ CKAN integration initialized")
    print(f"   URL: {CKAN_URL}")
    print(f"   API Key: {'configured' if ckan_config['api_key'] else 'not configured'}")

except Exception as e:
    print(f"❌ CKAN initialization error: {e}")

🌐 Initializing CKAN integration...
✅ CKAN integration initialized
   URL: http://ckan.tacc.cloud:5000
   API Key: not configured


In [72]:
# Publish campaign to CKAN using file upload
print(f"📊 Publishing campaign {campaign_id} to CKAN...")
try:
    # Get campaign data
    campaign_data = {
        'name': 'Environmental Monitoring Demo 2024',
        'description': 'Demonstration campaign for SDK usage and CKAN integration',
        'contact_name': 'Dr. Jane Smith',
        'contact_email': 'jane.smith@example.edu'
    }

    # Publish with file uploads
    ckan_result = ckan.publish_campaign(
        campaign_id=str(campaign_id),
        campaign_data=campaign_data,
        auto_publish=True,
        sensor_csv=str(sensors_csv),
        measurement_csv=str(measurements_csv)
    )

    print(f"✅ Campaign published to CKAN!")
    print(f"   Dataset ID: {ckan_result['dataset']['id']}")
    print(f"   Dataset Name: {ckan_result['dataset']['name']}")
    print(f"   CKAN URL: {ckan_result['ckan_url']}")
    print(f"   Resources created: {len(ckan_result['resources'])}")

    # Show resource details
    print(f"\n📎 Resources uploaded:")
    for resource in ckan_result['resources']:
        print(f"  • {resource['name']} ({resource['format']})")
        print(f"    Description: {resource['description']}")
        print(f"    Size: {resource.get('size', 'N/A')}")
        print()

except Exception as e:
    print(f"❌ CKAN publication error: {e}")

📊 Publishing campaign 609 to CKAN...


Failed to publish campaign to CKAN: Failed to create CKAN dataset: 403 Client Error: FORBIDDEN for url: http://ckan.tacc.cloud:5000/api/3/action/package_create


❌ CKAN publication error: CKAN publication failed: Failed to create CKAN dataset: 403 Client Error: FORBIDDEN for url: http://ckan.tacc.cloud:5000/api/3/action/package_create


In [73]:
# List CKAN datasets
print("📋 Listing CKAN datasets...")
try:
    datasets = ckan.list_datasets(
        tags=['environmental', 'upstream'],
        limit=10
    )

    print(f"Found {len(datasets)} datasets:")
    for dataset in datasets[:5]:  # Show first 5
        print(f"  • {dataset['name']}")
        print(f"    Title: {dataset['title']}")
        print(f"    Description: {dataset['notes'][:100]}...")
        print(f"    Resources: {len(dataset.get('resources', []))}")
        print()

except Exception as e:
    print(f"❌ Error listing datasets: {e}")

📋 Listing CKAN datasets...
Found 0 datasets:


## 6. Advanced Features

Let's demonstrate some advanced features like updating campaigns and stations, and working with CKAN organizations.

In [74]:
# Update campaign information
print(f"📝 Updating campaign {campaign_id}...")
try:
    updated_campaign = campaign_manager.update(
        campaign_id=str(campaign_id),
        description="Updated: Demonstration campaign for SDK usage and CKAN integration with advanced features"
    )

    print(f"✅ Campaign updated successfully!")
    print(f"   New description: {updated_campaign.description}")

except Exception as e:
    print(f"❌ Update error: {e}")

📝 Updating campaign 609...
❌ Update error: update() got an unexpected keyword argument 'description'


In [75]:
# Update station information
print(f"📝 Updating station {station_id}...")
try:
    updated_station = station_manager.update(
        station_id=str(station_id),
        campaign_id=str(campaign_id),
        description="Updated: Air quality monitoring station in downtown Austin with PM2.5 and PM10 sensors"
    )

    print(f"✅ Station updated successfully!")
    print(f"   New description: {updated_station.description}")

except Exception as e:
    print(f"❌ Update error: {e}")

📝 Updating station 479...
❌ Update error: update() got an unexpected keyword argument 'description'


In [76]:
# Work with CKAN organizations
print("🏢 Working with CKAN organizations...")
try:
    # List organizations
    organizations = ckan.list_organizations()

    print(f"Found {len(organizations)} organizations:")
    for org in organizations[:3]:  # Show first 3
        print(f"  • {org['name']}")
        print(f"    Title: {org['title']}")
        print(f"    Description: {org.get('description', 'N/A')[:80]}...")
        print(f"    Packages: {org.get('package_count', 'N/A')}")
        print()

except Exception as e:
    print(f"❌ Error working with organizations: {e}")

🏢 Working with CKAN organizations...
Found 0 organizations:


## 7. Error Handling and Validation

Let's demonstrate proper error handling and validation.

In [77]:
# Test validation errors
print("🧪 Testing validation and error handling...")

# Test invalid campaign creation
print("\n1. Testing invalid campaign creation:")
try:
    invalid_campaign = campaign_manager.create(
        name="",  # Empty name should fail
        description="Test campaign"
    )
except ValidationError as e:
    print(f"   ✅ Caught validation error: {e}")
except Exception as e:
    print(f"   ❌ Unexpected error: {e}")

# Test invalid station creation
print("\n2. Testing invalid station creation:")
try:
    invalid_station = station_manager.create(
        campaign_id=str(campaign_id),
        name="Test Station",
        latitude=100.0,  # Invalid latitude
        longitude=-97.7431
    )
except ValidationError as e:
    print(f"   ✅ Caught validation error: {e}")
except Exception as e:
    print(f"   ❌ Unexpected error: {e}")

# Test API errors
print("\n3. Testing API errors:")
try:
    # Try to get non-existent campaign
    nonexistent_campaign = campaign_manager.get("999999")
except APIError as e:
    print(f"   ✅ Caught API error: {e}")
except Exception as e:
    print(f"   ❌ Unexpected error: {e}")

🧪 Testing validation and error handling...

1. Testing invalid campaign creation:
   ❌ Unexpected error: create() got an unexpected keyword argument 'name'

2. Testing invalid station creation:
   ❌ Unexpected error: create() got an unexpected keyword argument 'name'

3. Testing API errors:
   ✅ Caught API error: Campaign not found: 999999


## 8. Data Retrieval and Analysis

Let's retrieve and analyze the data we've uploaded.

In [78]:
# Get campaign summary
print(f"📊 Campaign Summary for ID {campaign_id}:")
try:
    campaign_details = campaign_manager.get(str(campaign_id))
    stations_list = station_manager.list(campaign_id=str(campaign_id))

    print(f"\n📋 Campaign Information:")
    print(f"   Name: {campaign_details.name}")
    print(f"   Description: {campaign_details.description}")
    print(f"   Contact: {campaign_details.contact_name}")
    print(f"   Start Date: {campaign_details.start_date}")
    print(f"   End Date: {campaign_details.end_date}")

    print(f"\n📍 Stations ({stations_list.total} total):")
    for station in stations_list.items:
        print(f"   • {station.name} (ID: {station.id})")
        print(f"     Location: {station.latitude}, {station.longitude}")
        print(f"     Altitude: {station.altitude}m")
        print()

except Exception as e:
    print(f"❌ Error getting campaign summary: {e}")

📊 Campaign Summary for ID 609:

📋 Campaign Information:
   Name: Environmental Monitoring Demo 2024
   Description: Demonstration campaign for SDK usage and CKAN integration
   Contact: Dr. Jane Smith
   Start Date: 2025-07-17 09:07:26.136330
   End Date: 2026-07-17 09:07:26.136334

📍 Stations (1 total):
   • Downtown Air Quality Monitor (ID: 479)
❌ Error getting campaign summary: 'StationItemWithSummary' object has no attribute 'latitude'


## 9. Cleanup

Let's clean up by removing temporary files and logging out.

In [79]:
# Clean up temporary files
print("🧹 Cleaning up temporary files...")
try:
    if data_dir.exists():
        import shutil
        shutil.rmtree(data_dir)
        print(f"   ✅ Removed {data_dir}")
    else:
        print(f"   ℹ️  Directory {data_dir} does not exist")
except Exception as e:
    print(f"   ❌ Error cleaning up: {e}")

# Logout
print("\n👋 Logging out...")
try:
    client.logout()
    print("   ✅ Logged out successfully")
except Exception as e:
    print(f"   ❌ Logout error: {e}")

print("\n🎉 Demo completed successfully!")

🧹 Cleaning up temporary files...
   ✅ Removed sample_data

👋 Logging out...
   ✅ Logged out successfully

🎉 Demo completed successfully!


## Summary

This notebook demonstrated:

✅ **Authentication** - Secure login to the Upstream platform  
✅ **Campaign Management** - Creating, updating, and listing campaigns  
✅ **Station Management** - Setting up monitoring stations with coordinates  
✅ **Data Upload** - Uploading sensor and measurement data via CSV files  
✅ **CKAN Integration** - Publishing datasets to CKAN with file uploads  
✅ **Error Handling** - Proper validation and exception handling  
✅ **Data Retrieval** - Querying and analyzing uploaded data  

## Next Steps

- Explore additional sensor types and measurement formats
- Implement real-time data streaming
- Set up automated data processing pipelines
- Integrate with additional data portals
- Develop custom visualization dashboards

## Documentation

For more information, see:
- [Upstream SDK Documentation](https://upstream-sdk.readthedocs.io/)
- [CKAN API Documentation](https://docs.ckan.org/en/2.9/api/)
- [Environmental Data Standards](https://www.example.com/standards)

---

*This notebook was generated using the Upstream SDK v2.0*